In [2]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt

In [9]:
camera = cv2.VideoCapture(0)

h, w = None, None

model = load_model('face_mask_detection.h5')

network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yolov3.cfg',
                                     'yolo-coco-data/yolov3.weights')

layers_names_all = network.getLayerNames()
layers_names_output = \
[layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

probability_minimum = 0.5
threshold = 0.3

while True:

    _, frame = camera.read()

    if w is None or h is None:
        h, w = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (128, 128),
                                 swapRB=True, crop=False)

    network.setInput(blob)
    output_from_network = network.forward(layers_names_output)

    bounding_boxes = []
    confidences = []
    class_numbers = []
    
    for result in output_from_network:
        for detected_objects in result:

            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]

            if confidence_current > probability_minimum:
                box_current = detected_objects[0:4] * np.array([w, h, w, h]) 

                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)

    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)
    
    if len(results) > 0 :
        for i in results.flatten():
            if i==1:
                
                x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
                box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

                frame1 = frame[y_min:y_min + box_height, x_min:x_min + box_width]
                resized_img = cv2.resize(frame1, (128,128))
                normalized_img = resized_img/255.0
                reshaped_img = np.reshape(normalized_img,(1,128,128,3))
                result = model.predict(reshaped_img)
                predIdxs = np.argmax(result, axis = 1)[0]

                cv2.rectangle(frame, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              (0,255,255), 2)

                if predIdxs == 0:
                    mask = "No mask ({:.4f})".format(result[0][0])
                    cv2.putText(frame, mask, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                else:
                    nomask = "Mask ({:.4f})".format(result[0][1])
                    cv2.putText(frame, nomask, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            else:
                cv2.putText(frame, 'Please show your face properly', (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
                    
    else:
        continue

    cv2.namedWindow('Face Mask Detection', cv2.WINDOW_NORMAL)

    cv2.imshow('Face Mask Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-2b5g8ysb\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
